In [ ]:
import osmnx as ox
import numpy as np
import matplotlib.pyplot as plt
from typing import *
from tqdm import tqdm

In [ ]:
cities = ["Jilin_City", "Shanghai", "Paris", "Las_Vegas", "Tokyo"]
Point = Tuple[float, float]

In [ ]:
for city in cities:
    print(f"--- Processing {city} ---")
    graph = ox.load_graphml(f"{city}_street_network.graphml")
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(graph, nodes=True, edges=True)

    lines = gdf_edges['geometry']
    nodes_meta: Dict[Point, Set[int]]= dict()

    uid = 0
    for line in tqdm(lines, desc="Extracting nodes and edges"):
        # traverse all the points in the line
        for pi, pj in zip(line.coords[:-1], line.coords[1:]):

            if pi not in nodes_meta:
                nodes_meta[pi] = [uid, set()]
                uid += 1
            
            if pj not in nodes_meta:
                nodes_meta[pj] = [uid, set()]
                uid += 1

            # add the point pj to the neighbour of pi
            nodes_meta[pi][1].add(nodes_meta[pj][0])
            # add the point pi to the neighbour of pj
            nodes_meta[pj][1].add(nodes_meta[pi][0])

    nodes = np.array(list(nodes_meta.keys()))
    degrees = np.array([len(neighbours) for uid, neighbours in nodes_meta.values()])
    max_degree = np.max(degrees)
    edges = np.ones((len(nodes), max_degree), dtype=np.int32) - 1
    for i, (uid, neighbours) in enumerate(tqdm(nodes_meta.values(), desc="Converting edges to numpy array")):
        edges[i, :degrees[i]] = list(neighbours)

    print("Nodes shape:", nodes.shape)
    print("Edges shape:", edges.shape)
    print("Degrees shape:", degrees.shape)

    print("Saving to npz file\n")
    np.savez(f"{city}_street_network.npz", nodes=nodes, edges=edges, degrees=degrees)